# Recolección de datos

In [1]:
import requests
import pandas as pd

## Clasificación de países por nivel de ingreso

In [2]:
#Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
url = 'http://api.worldbank.org/v2/incomelevel' 
args = {"format":"json", "prefix":"Getdata"}
dic_Income_Level = requests.get(url, params=args)
datos = dic_Income_Level.json()[1]
data = pd.json_normalize(datos)
data[['value', 'id']]

,value,id
0,High income,HIC
1,Not classified,INX
2,Low income,LIC
3,Lower middle income,LMC
4,Low & middle income,LMY
5,Middle income,MIC
6,Upper middle income,UMC


In [3]:
incomes = ['HIC', 'UMC', 'LMC', 'LIC','INX']
trabajo = pd.DataFrame()
for income in incomes:
    url = 'http://api.worldbank.org/v2/country' #Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
    args = {"format":"json", "prefix":"Getdata", "incomelevel":income}
    dic_Income_Level = requests.get(url, params=args)
    datos = dic_Income_Level.json()[1]
    data = pd.json_normalize(datos)
    data = data[['name', 'id', 'region.value', 'incomeLevel.value']]
    trabajo = pd.concat([trabajo, data], ignore_index=True)


## Obtención de distintos indicadores

In [3]:
def consultar(pais: str, indicador:str, pagina:int = 1):

    '''Función que contacta a la API y devuelve la respuesta
    que brinda la misma en formato json'''

    # Página de la api y path al recurso solicitado
    api_url = 'http://api.worldbank.org/v2/es'
    path = f'/country/{pais}/indicator/{indicador}'
    url = api_url + path

    # Creamos el diccionario con los parametros 
    # para el método get
    args= {
        "date":'1990:2020',
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
   
    return requests.get(url,params=args).json()

In [4]:
def consultar_por_pais():
    '''Función que retorna un código de país,
    para, en caso que se quiera o se solicite,
    consultar en tiempo real la API'''
    paises=pd.read_parquet('../datasets/paises.parquet')

    seleccion = input("Ingrese un nombre de país: ")
    try:
        pais = paises.loc[paises.name.str.contains(seleccion, case=False),'id'].values[0]
    except:
        print("No se encontró el país, intente de nuevo")

In [4]:
# Estos indicadores fueron seleccionados 
# de forma manual al final del documento
indicadores = pd.read_csv('../datasets/indicadores_seleccionados.csv')

tuplas = [(a,b) for a,b in zip(indicadores.id.to_list(), 
          indicadores.name.to_list())]

In [12]:
def carga_incremental(pais, indicador):
    '''Función que a partir de un país y un indicador 
    llama a consultar y establece qué tipo de contenido tiene
    según eso devuelve o no un dataframe con todos los datos'''

    consulta = consultar(pais, indicador)
    try:
        # La primera parte de la respuesta nos indica en 
        # cuantas páginas de encuentra la información
        paginas = consulta[0]["pages"]

        # La segunda parte nos retorna una lista de 
        # diccionarios con la información que queríamos
        datos=consulta[1]

    except:
        print('No hay datos para:', indicador, pais)
        pass
    else:
        if paginas >= 1:
            # Agregamos los valores de las otras páginas a
            # nuestra lista de diccionarios
            for pagina in range(2,paginas+1):
                datos.extend(consultar(pais, indicador, pagina)[1])

            # Creo el DataFrame con todos los datos
            data = pd.json_normalize(datos)
            return data
        return pd.DataFrame(['error'],columns=['no_data'])


In [ ]:
for indicador, _ in tuplas:
    print(indicador)
    datos = carga_incremental(pais='all', indicador=indicador)

    # Guardo el dataframe resultante
    datos.to_parquet(f'../datasets/df_TWB_{indicador}.parquet')
    print('Guardado')


# ETL

## Búsqueda de indicadores

La lista de indicadores bajo estudio se selecciona de forma manual a partir de los datos que indican distintas tesis de estudio. De todas formas los indicadores pasarán por un tamiz de análisis exploratorio

In [19]:
indicators = pd.read_csv('../datasets/indicadores.csv')

# Se completó los valores faltantes con 'na'
# para poder buscar sobre el dataframe
indicators.fillna('na',inplace=True)

In [37]:
interes = input("Indicador de interés: ")
indicators.loc[indicators.name.str.contains(interes,case=False),
     ['id','name','sourceNote']]#.to_csv('temp.csv')

,id,name,sourceNote
17810,SP.POP.TOTL.FE.IN,"Población, mujeres","Población, mujeres"
17811,SP.POP.TOTL.FE.ZS,"Población, mujeres (% del total)",La población femenina es el porcentaje de la p...


In [26]:
interes = input("Indicador de interés: ")
pick = indicators.loc[indicators.id == interes,
     ['id','name','sourceNote']]
pick

,id,name,sourceNote
17815,SP.POP.TOTL.MA.ZS,"Población, hombres (% del total)",na


In [108]:
trabajo = pd.concat([trabajo, pick], ignore_index=True)

In [115]:
trabajo.to_csv('../datasets/indicadores_seleccionados.csv')

## Exploración de los Datasets obtenidos

In [34]:
codigos = [i for i, j in tuplas]
nombres = [j for i, j in tuplas]

columnas=['esperanza_vida_total', 'esperanza_vida_mujeres','esperanza_vida_varones','índice_gini','gasto_púb_educacion_pje','duración_educ_obligatoria','pib_pc_usd_actuales','pib_ppa_prec_inter','capacidad_estadística','población_mujeres_pje','población_hombres_pje','pib_pc_prec_inter']
print(len(columnas))
print(len(nombres))

holiwis


In [72]:
import os
directorio = '../datasets/'

with os.scandir(directorio) as ficheros:

    trabajo=pd.read_parquet('../datasets/df_TWB_SP.DYN.LE00.IN.parquet')[['date','countryiso3code']]

    for fichero in ficheros:
        if fichero.name.startswith('df_TWB'):

            # obtengo el cógigo de indicador que se encuentra en el nombre del fichero
            codigo_fichero = fichero.name[7:-8]
            # busco el código en mi lista de códigos 
            # y procedo a renombrar la columna de interés
            if codigo_fichero in codigos:
                df=pd.read_parquet(directorio+fichero.name)

                # obtengo el índice en mi lista de códigos que 
                # se corresponde con el índice en mi lista de nombres
                indice_codigo = codigos.index(codigo_fichero)
                #df.rename(columns={'value':columnas[indice_codigo]},inplace=True)
                trabajo[columnas[indice_codigo]]=df.value
                
                

In [73]:
trabajo

,date,countryiso3code,pib_ppa_prec_inter,población_mujeres_pje,esperanza_vida_total,población_hombres_pje,duración_educ_obligatoria,índice_gini,esperanza_vida_mujeres,esperanza_vida_varones,gasto_púb_educacion_pje,capacidad_estadística,pib_pc_prec_inter,pib_pc_usd_actuales
0,2020,AFE,2.478218e+12,50.364579,64.325702,49.635421,7.0,NaN,66.534628,62.121491,4.608170,57.632861,3659.272868,1360.878645
1,2019,AFE,2.517975e+12,50.373008,64.005213,49.626992,7.0,NaN,66.202116,61.813185,4.846845,58.502425,3814.846625,1511.309259
2,2018,AFE,2.418685e+12,50.382358,63.648988,49.617642,7.0,NaN,65.832944,61.470354,4.951635,58.067643,3761.035160,1541.031661
3,2017,AFE,2.299420e+12,50.392615,63.246264,49.607385,7.0,NaN,65.415515,61.083445,4.963090,59.710148,3670.891196,1629.404273
4,2016,AFE,2.212683e+12,50.403727,62.787681,49.596273,7.0,NaN,64.939294,60.643170,4.821875,58.599045,3627.474284,1446.533624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8241,1994,ZWE,2.165871e+10,50.486345,52.044000,49.513655,NaN,NaN,55.313000,49.136000,44.333981,NaN,1923.209711,611.865276
8242,1993,ZWE,1.941304e+10,50.412002,53.653000,49.587998,NaN,NaN,56.995000,50.663000,NaN,NaN,1750.061859,591.719682
8243,1992,ZWE,1.876622e+10,50.343007,55.243000,49.656993,NaN,NaN,58.608000,52.210000,22.322210,NaN,1721.591128,619.372083
8244,1991,ZWE,2.016618e+10,50.285057,56.749000,49.714943,NaN,NaN,60.086000,53.717000,NaN,NaN,1888.041180,809.051140
